<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install firebase



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
#@title
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from ipywidgets import VBox, HBox, HTML

questionsFromDB = []
# display(IPython.display.Markdown(f"**Points:** {points}"))


points = 0
quantity_of_questions=0
quantity_of_correct_questions=0

currentSelectedAns = ""
finalSelectedBtn = widgets.Button(description="") # null object
question = ""
randomQuestionAndAnswer = NULL
answers = []
buttonsLst = []
questionsHistory = set() # saving the 'old' questions for non repeating. Can also be done by removing those question from all DB list (but i think it's not preferred)


def get_all_database():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication('https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/',None)
  res = FBconn.get('/Questions/',None)

  if res:
    for key in res:
      if key is not None: 
        questionsFromDB.append(key)

# Define a function to save the selected answer

def get_question_and_answers_dictionary():
        global randomQuestionAndAnswer, answers, question, radioButtonLst

        randomQuestionAndAnswer = NULL # we want to generate new item...
        if (questionsFromDB):
          while randomQuestionAndAnswer == NULL:
            randomQuestionAndAnswer = random.choice(questionsFromDB)
            if str(randomQuestionAndAnswer['question']) in questionsHistory:
              randomQuestionAndAnswer = NULL
        else:
          return

        if randomQuestionAndAnswer != NULL:            
          question = str(randomQuestionAndAnswer['question'])
          questionsHistory.add(question)
          answers = [str(ans) for ans in randomQuestionAndAnswer['answers']]
          answers = answers[1:]

# on_click of answer button
def save_answer(b):
  global currentSelectedAns, finalSelectedBtn
  currentSelectedAns = str(b.description)  # save the answer to compare
  finalSelectedBtn = b  # save the button object to mark it later
  on_answer_button_clicked(b)
  update_total_questions_quantity()

def create_and_show_elements():
  global randomQuestionAndAnswer, answers, question, radioButtonLst
  display(IPython.display.HTML(f"<h2>The question is: {question}</h2>"))
  # display(IPython.display.Markdown(f"**Points:** {points}"))
  # correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
  # total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
  # points_label = IPython.display.Markdown(f"**Points:** {points}")
  points_label = HTML(f"<strong>Points:</strong> {points}")
  correct_label = HTML(f"<strong>Correct Questions:</strong> {quantity_of_correct_questions}")
  total_label = HTML(f"<strong>Out Of:</strong> {quantity_of_questions}")

  display(HBox([points_label, correct_label, total_label]))

  buttonsLst = [widgets.Button(description = answer, layout = widgets.Layout( width='auto', height='auto')  ) for answer in answers]

  for btn in buttonsLst:
    btn.on_click(save_answer)
    btn.style.button_color = 'orange'

  left_box = VBox([buttonsLst[0], buttonsLst[1]])
  right_box = VBox([buttonsLst[2], buttonsLst[3]])
  display(HBox([left_box, right_box]))


# refresh view -> get new question
def generate_new_question():
  output.clear()
  get_question_and_answers_dictionary()
  create_and_show_elements()
  # update_total_questions_quantity()
  
def on_answer_button_clicked(b):
    global finalSelectedBtn
    # iterate over question widgets and get user answers
    answerId = randomQuestionAndAnswer['correctAnswer']
    if answerId:
      correctAns = answers[int(answerId)-1]

    if currentSelectedAns is NULL:
      print("Please select answer")
      return

   
    # check if user's answer is correct
    if currentSelectedAns.strip() == correctAns.strip():
        update_points(3)
        update_total_correct_questions_quantity()
        print("Correct!")
        finalSelectedBtn.style.button_color = 'lightgreen'
        # start timer and refresh
        time.sleep(2)
        generate_new_question()
    else:
        update_points(-1)
        finalSelectedBtn.style.button_color = 'red'
        display(IPython.display.HTML('<br/><h4>Incorrect answer<br/>Try Again! You can do it!<h4/><br/>'))




# Update points and display them
def update_points(new_points):
    global points
    points += new_points
    display(IPython.display.Markdown(f"**New Points:** {points}"))
    correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
    total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
    display(correct_label)
    display(total_label)

def update_total_questions_quantity():
    global quantity_of_questions
    quantity_of_questions+=1

def update_total_correct_questions_quantity():
    global quantity_of_correct_questions
    quantity_of_correct_questions+=1



def create_menu_buttons():
    btn_questions = widgets.Button(description="Questions")
    btn_classes = widgets.Button(description="Classes")
    btn_help = widgets.Button(description="Help")


    def on_questions_clicked(b):
        output.clear()
        print("Questions Page")
        generate_new_question()
    
    def on_classes_clicked(b):
        output.clear()
        print("Classes Page")
    
    def on_help_clicked(b):
        output.clear()
        print("Help Page")

    # Assign callbacks to buttons
    btn_questions.on_click(on_questions_clicked)
    btn_classes.on_click(on_classes_clicked)
    btn_help.on_click(on_help_clicked)

    # Display Menu buttons
    display(VBox([btn_questions, btn_classes, btn_help]))


# Define callbacks
def on_start_clicked(b):
    output.clear()
    create_menu_buttons()
    

def start_game():
    # Create buttons
    btn_start = widgets.Button(description="Start Now")
    display(btn_start)
    btn_start.on_click(on_start_clicked)

  
##### MAIN #####


get_all_database()
start_game()
# generate_new_question()

# submit_button = widgets.Button(description="Submit") # prompt user to submit answers
# submit_button.on_click(on_submit_button_clicked) # assign callback function to submit button

# def on_clicked(event):
#     print("A click event occurred:", event)

# output.register_callback('notebook.onClicked', on_clicked)

